In [1]:
import xgboost as xgb
import numpy as np
import pandas as pd# Used for Regression Modelling
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle

In [26]:
data = pd.read_csv('./data/merged_mldata2020v2.csv',delimiter = ',')
with open(f'score_predictor.pkl', 'rb') as f:
    model = pickle.load(f)

outputs = []

games = data.drop(columns =['vscore', 'hscore','winner','ou',"sWinner","fspread","fscore","vospread","vcspread"])
games = pd.DataFrame(games)
weekfilter = games['week'] == 1

games = games[weekfilter]
#print(games)
#columnheads = ['temp','wind_mph','vdflg','hdflg','divgame','nsite','hospread','ouopen','hcspread','ouclose','vTOTAL.DVOA','vTOTAL.RNK','vOFF.RNK','vOFF.DVOA','vDEF.RNK','vDEF.DVOA','vST.RNK','vST.DVOA','hTOTAL.DVOA','hTOTAL.RNK','hOFF.RNK','hOFF.DVOA','hDEF.RNK','hDEF.DVOA','hST.RNK','hST.DVOA','vtsw','vtsl','vtst','htsw','htsl','htst','vtw','vtl','vtt','vts','htw','htl','htt','hts']

for i in games.iterrows():
    #prediction = {'vteam':i['vteam'],'hteam':i['hteam']}
    #input_variables = x.drop(columns=['week','vteam','hteam'])
    #prediction['outcome'] = model.predict(input_variables)[0]
    print(i['vteam'])


#print(x)
#print(teams)

TypeError: tuple indices must be integers or slices, not str